In [ ]:
!pip install astroNN

In [ ]:
#importing required libraries
import numpy as np
import keras

from keras.models import Sequential
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten, Dropout
from keras.optimizers import Adam
from astroNN.datasets import galaxy10
from astroNN.datasets.galaxy10 import galaxy10cls_lookup
from sklearn.metrics import classification_report ,confusion_matrix
from tensorflow.keras import utils

In [4]:
def preprocess(images):
    """ Normalize the images
    Parameter
    ---------
    images : list of images
        The images to normalize
    Return
    -------
        Normalized images
    """
    images = images/255

    return images

In [5]:
#loading the dataset
images, labels = galaxy10.load_data()

Galaxy10.h5:  94%|█████████▍| 199M/210M [00:02<00:00, 97.9MB/s]

Downloaded Galaxy10 successfully to /root/.astroNN/datasets/Galaxy10.h5


Galaxy10.h5: 210MB [00:02, 72.1MB/s]                           


In [5]:
# To convert the labels to categorical 10 classes
labels = utils.to_categorical(labels, 10)

#converting the arrays to float type
labels = labels.astype(np.float32)
images = images.astype(np.float32)

In [6]:
#normalizing images
images = preprocess(images)

In [8]:
X = images
y = labels

In [9]:
#splitting the data into train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)

In [16]:
#building a seq2seq model
model = Sequential()

model.add(Conv2D(32, kernel_size=(3,3),activation='relu', input_shape=(69,69,3)))
model.add(Conv2D(filters=32,kernel_size=(3,3),padding='Same',activation='relu', kernel_initializer = 'he_uniform'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=64,kernel_size=(3,3),padding='Same',activation='relu', kernel_initializer = 'he_uniform'))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding='Same',activation='relu', kernel_initializer = 'he_uniform'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=128,kernel_size=(3,3),padding='Same',activation='relu', kernel_initializer = 'he_uniform'))
model.add(Conv2D(filters=128,kernel_size=(3,3),padding='Same',activation='relu', kernel_initializer = 'he_uniform'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=128,kernel_size=(3,3),padding='Same',activation='relu', kernel_initializer = 'he_uniform'))
model.add(Conv2D(filters=128,kernel_size=(3,3),padding='Same',activation='relu', kernel_initializer = 'he_uniform'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=256,kernel_size=(3,3),padding='Same',activation='relu', kernel_initializer = 'he_uniform'))
model.add(Conv2D(filters=256,kernel_size=(3,3),padding='Same',activation='relu', kernel_initializer = 'he_uniform'))
model.add(Conv2D(filters=256,kernel_size=(3,3),padding='Same',activation='relu', kernel_initializer = 'he_uniform'))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Flatten())
model.add(Dense(128, activation='relu', kernel_initializer = 'he_uniform'))
model.add(Dense(128, activation='relu', kernel_initializer = 'he_uniform'))
model.add(Dense(128, activation='relu', kernel_initializer = 'he_uniform'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

#defining the batch sizes and epochs
batch_size = 32
epochs = 30

In [17]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [18]:
early_stop= EarlyStopping(monitor='val_loss',patience=2)

In [19]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 67, 67, 32)        896       
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 67, 67, 32)        9248      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 33, 33, 32)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 33, 33, 64)        18496     
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 33, 33, 64)        36928     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 16, 16, 128)      

In [20]:
#training the model
history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(X_test, y_test))

Epoch 1/30
613/613 [==============================] - 14s 20ms/step - loss: 1.9429 - accuracy: 0.3063 - val_loss: 1.3012 - val_accuracy: 0.3809
Epoch 2/30
613/613 [==============================] - 12s 20ms/step - loss: 1.2850 - accuracy: 0.4712 - val_loss: 1.0636 - val_accuracy: 0.5737
Epoch 3/30
613/613 [==============================] - 12s 20ms/step - loss: 0.9457 - accuracy: 0.6603 - val_loss: 1.1737 - val_accuracy: 0.6145
Epoch 4/30
613/613 [==============================] - 12s 20ms/step - loss: 0.8305 - accuracy: 0.7152 - val_loss: 0.8352 - val_accuracy: 0.7159
Epoch 5/30
613/613 [==============================] - 12s 20ms/step - loss: 0.7886 - accuracy: 0.7286 - val_loss: 0.7613 - val_accuracy: 0.7352
Epoch 6/30
613/613 [==============================] - 12s 20ms/step - loss: 0.7803 - accuracy: 0.7394 - val_loss: 0.7231 - val_accuracy: 0.7559
Epoch 7/30
613/613 [==============================] - 12s 20ms/step - loss: 0.7522 - accuracy: 0.7484 - val_loss: 0.8142 - val_accuracy:

In [21]:
#evaluating model over test dataset
model.evaluate(X_test, y_test)

69/69 [==============================] - 1s 7ms/step - loss: 0.8931 - accuracy: 0.7223


[0.8931226134300232, 0.7223497033119202]

In [22]:
#saving the model
model.save("galaxy.h5")

In [38]:
#displaying classification report
y_pred=model.predict_classes(X_test)
y_pred

#converting the one-hot encoded labels back
rounded_labels=np.argmax(y_test, axis=1)

print('Classification Report')
print(classification_report(rounded_labels,y_pred))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Classification Report
              precision    recall  f1-score   support

           0       0.47      0.32      0.38       360
           1       0.84      0.96      0.89       703
           2       0.73      0.92      0.82       599
           3       0.00      0.00      0.00        35
           4       0.60      0.94      0.73       151
           5       0.00      0.00      0.00         3
           6       0.88      0.33      0.48        64
           7       0.92      0.10      0.17       126
           8       0.55      0.38      0.45        90
           9       0.61      0.42      0.49        48

    accuracy                           0.72      2179
   macro avg       0.56      0.44      0.44      2179
weighted avg       0.71      0.72      0.68      2179



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [39]:
#displaying confusion matrix
print('Confusion Matrix')
print(confusion_matrix(rounded_labels,y_pred))

Confusion Matrix
[[116  77 143   0  10   0   0   0  12   2]
 [  5 676  22   0   0   0   0   0   0   0]
 [  5  39 553   0   2   0   0   0   0   0]
 [  2   0   2   0  31   0   0   0   0   0]
 [  1   0   5   0 142   0   3   0   0   0]
 [  0   0   0   0   3   0   0   0   0   0]
 [  1   0   0   0  42   0  21   0   0   0]
 [ 64  11  25   0   2   0   0  12  11   1]
 [ 36   3   5   0   1   0   0   1  34  10]
 [ 16   2   2   0   3   0   0   0   5  20]]


In [32]:
#displaying the predictions along with the actual values
for i in range(10):

    img = images[i]

    img = np.expand_dims(img,0) # <--- add batch axis
    #print(img.shape)
    output = model.predict(img)
    #print(output[0])
    predictedClass = np.argmax(output[0])

    print("\nPredicted: ",galaxy10cls_lookup(predictedClass))
    print("Actual: ",galaxy10cls_lookup(np.argmax(labels[i])))
    print("-------------------------------------------------------------------------------------------------------------------------------------------")


Predicted:  Smooth, in-between round
Actual:  Smooth, in-between round
-------------------------------------------------------------------------------------------------------------------------------------------

Predicted:  Smooth, in-between round
Actual:  Smooth, in-between round
-------------------------------------------------------------------------------------------------------------------------------------------

Predicted:  Disk, Edge-on, Rounded Bulge
Actual:  Disk, Edge-on, Rounded Bulge
-------------------------------------------------------------------------------------------------------------------------------------------

Predicted:  Smooth, in-between round
Actual:  Smooth, in-between round
-------------------------------------------------------------------------------------------------------------------------------------------

Predicted:  Disk, Edge-on, Rounded Bulge
Actual:  Disk, Edge-on, Rounded Bulge
----------------------------------------------------------------